In [1]:
import random
import collections
import math
import sys
import copy
import pandas as pd
import os
import datetime
import numpy as np

In [2]:
fake = pd.read_csv(os.getcwd()+'/fake.csv')
fake = fake[['user_key','text','created_str']]

In [3]:
real = pd.read_csv(os.getcwd()+'/real.csv')
real = real[['created','name','text']]
real['truth'] = 1
fake['truth'] = -1

fake = fake.rename(columns={"created_str": "created", "user_key": "name", })
real['created'] = pd.to_datetime(real['created'])
fake['created'] = pd.to_datetime(fake['created'])

real['month'] = real['created'].dt.month
fake['month'] = fake['created'].dt.month
real['day'] = real['created'].dt.day
fake['day'] = fake['created'].dt.day
real['hr'] = real['created'].dt.hour
fake['hr'] = fake['created'].dt.hour

In [4]:
real.head()

,created,name,text,truth,month,day,hr
0,2013-12-05 01:48:00,sheezy0,Robbie E Responds To Critics After Win Against...,1,12,5,1
1,2012-10-01 13:51:00,DavdBurnett,‰ЫПIt felt like they were my friends and I was...,1,10,1,13
2,2014-11-28 11:30:00,lwtprettylaugh,i absolutely adore when louis starts the songs...,1,11,28,11
3,2009-06-11 22:39:00,douggarland,Hi @JordanSpieth - Looking at the url - do you...,1,6,11,22
4,2014-04-16 13:23:00,WilfordGemma,Watching Neighbours on Sky+ catching up with t...,1,4,16,13


In [5]:
fake.head()

,name,text,created,truth,month,day,hr
0,ryanmaxwell_1,#IslamKills Are you trying to say that there w...,2016-03-22 18:31:42,-1,3.0,22.0,18.0
1,detroitdailynew,"Clinton: Trump should’ve apologized more, atta...",2016-10-10 20:57:00,-1,10.0,10.0,20.0
2,cookncooks,RT @ltapoll: Who was/is the best president of ...,2017-02-22 12:43:43,-1,2.0,22.0,12.0
3,queenofthewo,RT @jww372: I don't have to guess your religio...,2016-12-26 15:06:41,-1,12.0,26.0,15.0
4,mrclydepratt,RT @Shareblue: Pence and his lawyers decided w...,2017-08-06 02:36:24,-1,8.0,6.0,2.0


In [6]:
real.shape

(20050, 7)

In [7]:
fake.shape

(203482, 7)

In [ ]:
train_real = real.head(10000)
train_fake = fake.head(10000)
train = train_real.append(train_fake, sort=True)
train = train.sample(frac=1).reset_index(drop=True)

test_real = real.tail(1000)
test_fake = fake.tail(1000)
test = test_real.append(test_fake, sort=True)
test = test.sample(frac=1).reset_index(drop=True)

trainExamples = []

for index, row in train.iterrows():
    trainExamples.append((row['text'], row['truth']))
trainExamples = tuple(trainExamples)

testExamples = []

for index, row in test.iterrows():
    testExamples.append((row['text'], row['truth']))
testExamples = tuple(testExamples)

In [9]:
# trainExamples

In [10]:
def extractWordFeatures(x):
    features = {}
    for word in x.split():
        if word not in features:
            features[word] = 0
        features[word] += 1
    return features

In [11]:
def learnPredictor(trainExamples, testExamples, featureExtractor, numIters, eta):
    def hingeLoss(w, i):
        x, y = trainExamples[i]
        features = featureExtractor(x)
        return max(1 - dotProduct(weights,features)*y, 0)
        
    def dHingeLoss(w, i):
        x, y = trainExamples[i]
        features = featureExtractor(x)
        if dotProduct(weights,features)*y < 1:
            features.update((a, b*-y) for a, b in features.items())
            return features
        else:
            features.update((a, 0) for a, b in features.items())
            return features
        
    weights = {}

    n = len(trainExamples)
    for t in range(numIters):
        for i in range(n):
            gradient = dHingeLoss(weights, i)
            increment(weights, -eta, gradient)
    predictor = lambda x : 1 if dotProduct(featureExtractor(x), weights) > 0 else -1
    print('Train Error: {}'.format(evaluatePredictor(trainExamples, predictor)))
    print('Test Error: {}'.format(evaluatePredictor(testExamples, predictor)))
    trainPredictions = predictions(trainExamples, predictor)
    testPredictions = predictions(testExamples, predictor)
    return weights, trainPredictions, testPredictions

In [12]:
def combinedFeatures(x):
    features = {}
    for word in x.split():
        if word not in features:
            features[word] = 0
        features[word] += 1
    
    n = 2
    x = x.replace(" ","")
    for i in range(len(x) - n + 1):
        if x[i:i+n] not in features:
            features[x[i:i+n]] = 0
        features[x[i:i+n]] += 1
    return features

In [13]:
def extractCharacterFeatures(n):
    def extract(x):
        x = x.replace(" ","")
        nGrams = {}
        for i in range(len(x) - n + 1):
            if x[i:i+n] not in nGrams:
                nGrams[x[i:i+n]] = 0
            nGrams[x[i:i+n]] += 1
        return nGrams
    return extract

In [14]:
def dotProduct(d1, d2):
    if len(d1) < len(d2):
        return dotProduct(d2, d1)
    else:
        return sum(d1.get(f, 0) * v for f, v in list(d2.items()))
    
def increment(d1, scale, d2):
    for f, v in list(d2.items()):
        d1[f] = d1.get(f, 0) + v * scale
        
def evaluatePredictor(examples, predictor):
    error = 0
    for x, y in examples:
        if predictor(x) != y:
            error += 1
    return 1.0 * error / len(examples)

def predictions(examples, predictor):
    predictions = []
    for x, y in examples:
        predictions.append(predictor(x))
    return predictions

In [15]:
featureExtractor = combinedFeatures
weightsCombined, trainPredictionsCombined, testPredictionsCombined = learnPredictor(trainExamples, testExamples, featureExtractor, numIters=20, eta=0.01)

Train Error: 0.0032
Test Error: 0.0965


In [16]:
featureExtractor = extractWordFeatures
weightsWords, trainPredictionsWords, testPredictionsWords = learnPredictor(trainExamples, testExamples, featureExtractor, numIters=20, eta=0.01)

Train Error: 0.00705
Test Error: 0.2115


In [17]:
featureExtractor = extractCharacterFeatures(3)
weightsCharacters3, trainPredictionsCharacters3, testPredictionsCharacters3 = learnPredictor(trainExamples, testExamples, featureExtractor, numIters=20, eta=0.01)

Train Error: 0.00045
Test Error: 0.0995


In [18]:
featureExtractor = extractCharacterFeatures(4)
weightsCharacters4, trainPredictionsCharacters4, testPredictionsCharacters4 = learnPredictor(trainExamples, testExamples, featureExtractor, numIters=20, eta=0.01)

Train Error: 0.0
Test Error: 0.13


In [19]:
featureExtractor = extractCharacterFeatures(2)
weightsCharacters2, trainPredictionsCharacters2, testPredictionsCharacters2 = learnPredictor(trainExamples, testExamples, featureExtractor, numIters=20, eta=0.01)

Train Error: 0.0163
Test Error: 0.089


In [22]:
def addFeatures(df, dataType):
    if dataType == 'train':
        df['characters2'] = trainPredictionsCharacters2
        df['characters4'] = trainPredictionsCharacters4
        df['characters3'] = trainPredictionsCharacters3
        df['combined'] = trainPredictionsCombined
        df['words'] = trainPredictionsWords
    else:
        df['characters2'] = testPredictionsCharacters2
        df['characters4'] = testPredictionsCharacters4
        df['characters3'] = testPredictionsCharacters3
        df['combined'] = testPredictionsCombined
        df['words'] = testPredictionsWords
    df = addNumbersInName(df)
    df = addSpecialCharactersInText(df)
    df = addCapsInText(df)
    return df

In [23]:
def addNumbersInName(df):
    numbersInName = []
    for index, row in df.iterrows():
        count = 0
        text = row['name']
        for i in range(10):
            count += text.count(str(i))
        numbersInName.append(count)
    df['numbersInName'] = numbersInName
    return df

In [40]:
def addSpecialCharactersInText(df):
    specialCharsText = []
    for index, row in df.iterrows():
        count = 0
        text = row['text']
        for ch in text:
            if not ((ch >= 'A' and ch <= 'Z') or (ch >= 'a' and ch <= 'z')):
                count += 1
        specialCharsText.append(1.0*count/len(text))
    df['specialCharsText'] = specialCharsText
    return df

In [41]:
def addCapsInText(df):
    capsInText = []
    for index, row in df.iterrows():
        count = 0
        text = row['text']
        for ch in text:
            if ch >= 'A' and ch <= 'Z':
                count += 1
        capsInText.append(1.0*count/len(text))
    df['capsInText'] = capsInText
    return df

In [42]:
newTrain = addFeatures(train, 'train')
newTest = addFeatures(test, 'test')

In [43]:
newTrain.to_csv('newTrain.csv')
newTest.to_csv('newTest.csv')

In [47]:
newTrain.head()

,created,day,hr,month,name,text,truth,characters2,characters4,characters3,combined,words,numbersInName,specialCharsText,capsInText
0,2013-08-14 12:54:00,14.0,12.0,8.0,karenhbullard,@SydnieJR except once the Hallmark movies star...,1,1,1,1,1,1,0,0.318182,0.056818
1,2010-07-04 13:08:00,4.0,13.0,7.0,amajorpain,Shaw vows to 'work across the aisle' to figh h...,1,1,1,1,1,1,0,0.242647,0.102941
2,2016-12-28 16:04:36,28.0,16.0,12.0,melvinsroberts,RT @rashawnleo91: #2016In4Words GAYS RULED THE...,-1,-1,-1,-1,-1,-1,0,0.315789,0.368421
3,2009-06-09 07:54:00,9.0,7.0,6.0,Spandavia,@SMEAllianceLtd Isn't that the pension scheme ...,1,1,1,1,1,1,0,0.173611,0.062500
4,2010-03-17 08:21:00,17.0,8.0,3.0,AmnahThunyan,The only difference between pointers of differ...,1,1,1,1,1,1,0,0.190909,0.063636


In [45]:
train_x = newTrain.drop(columns=['truth','created','name','text'])

In [46]:
train_x['day'] = train_x['day'].astype(int)
train_x['hr'] = train_x['hr'].astype(int)
train_x['month'] = train_x['month'].astype(int)

In [31]:
train_x.dtypes

day                   int64
hr                    int64
month                 int64
characters2           int64
characters4           int64
characters3           int64
combined              int64
words                 int64
numbersInName         int64
specialCharsText    float64
capsInText          float64
dtype: object

In [32]:
train_y = newTrain[['truth']]

In [33]:
train_x.head()

,day,hr,month,characters2,characters4,characters3,combined,words,numbersInName,specialCharsText,capsInText
0,14,12,8,1,1,1,1,1,0,1.0,0.056818
1,4,13,7,1,1,1,1,1,0,1.0,0.102941
2,28,16,12,-1,-1,-1,-1,-1,0,1.0,0.368421
3,9,7,6,1,1,1,1,1,0,1.0,0.062500
4,17,8,3,1,1,1,1,1,0,1.0,0.063636


In [34]:
train_y.head()

,truth
0,1
1,1
2,-1
3,1
4,1
